Собираем корпус данных (через Colab) с помощью Selenium.

Задача проекта - получить данные по сегментации составных японских единиц с использованием различных словарей. В связи с этим решила парсить контексты с составными единицами, полученные в результате поиска по объемному корпусу японских текстов, поскольку такой вариант позволяет лучше контролировать поиск необходимых данных.

Проект представляет собой расширение более раннего исследования, для которого типы составных единиц были заранее составлены на основе ряда научных публикаций на тему сложных единиц в японском языке, а списки составных единиц - при помощи случайного выбора с использованием корпуса BCCWJ ("Сбалансированный корпус письменных текстов японского языка"). В ходе выполнения данного проекта некоторые списки пополнялись ввиду необходимости проверить появляющиеся гипотезы. В тетради представлены итоговые списки составных единиц, включая те, контексты с которыми найдены не были. В дальнейшем планирую еще расширить исследование за счет включения большего числа типов составных единиц и расширения списка имеющихся.

Нужные для поиска списки составных единиц для удобства положила в словарь. С помощью Selenium заходим на сайт, вводим пароль и имя пользователя (пользовалась своими, из данной тетрадки по понятным причинам их убрала). На сайте представлено несколько корпусов, я решила воспользоваться наиболее объемным и обработанным (BCCWJ), в дальнейшем можно поискать и в других. Зайдя в корпус BCCWJ, выбираем вкладку поиска по LUW (единственный вариант для составных единиц), указываем поиск по 語彙素 (близок термину "лексема"). Затем идем по словарю единиц для поиска, поочередно вводим каждое из них в графу поиска, нажимаем кнопку "Поиск" и парсим нужную информацию. После сохранения контекстов в файл очищаем графу поиска, выжидаем некоторое время, чтобы не перегрузить сайт, и переходим к следующей составной единице.

Выдача результатов поиска в BCCWJ представляет собой таблицу с 500 случайно отобранными контекстами (если найденных контекстов меньше 500, выдаются просто все найденные), в которой указана самая разная информация о единице и контексте с ней. Для составления корпуса данных для проекта я брала три столбца: левый контекст, ключевое слово (в данном случае составная единица в определенной форме, которая представлена в контексте) и правый контекст. По каким-то причинам pandas не смог скопировать таблицу, и я воспользовалась Selenium. В найденные данные попадали и сами фразы "левый контекст", "ключевое слово" и "правый контекст" из шапки таблицы, поэтому я воспользовалась сравнением строк, чтобы убрать лишнее. 

Контекст воссоздавался с помощью склеивания данных следующим образом. Если составная единица представляла собой глагол-ваго, то контекст склеивался так: левый контекст + единица для поиска без последнего символа + последний символ ключевого слова + правый контекст, это обусловлено тем, что, во-первых, глаголы-ваго в японском языке спрягаются, у них может меняться последний символ (следовательно, нужно сохранить последний символ ключевого слова), и, во-вторых, написание одной и той же единицы в японском может быть разным, а для проекта нам было необходимо единообразное написание составной единицы во всех найденных с ней контекстах (следовательно, берем составную единицу без последнего меняющего символа). В остальных случаях достаточно было сохранить единообразное написание единицы во всех контекстах с ней, поэтому контекст склеивался так: левый контекст + единица для поиска + правый контекст.

Дополнительно, для изучения сегментации единиц в зависимости от варианта их написания, я создала файлы с контекстами на основе уже полученных, заменив с помощью регулярных выражений один вариант написания составной единицы на другой.

In [ ]:
%pip install google-colab-selenium

In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

In [ ]:
import pandas as pd
import os, os.path as osp
import shutil
import re

In [ ]:
# составные единицы группы канго с "постфиксами"

kango_with_postfixes = [
    '図書室', '事務室', '研究室', '実験室', '保健室', '新聞室', '美容室', '喫茶室', '調理室', '冷蔵室',
    '密室', '茶室', '教室', '正室', '側室', '客室', '個室', '自室', '病室', '皇室',
    '図書館', '体育館', '映画館', '大使館', '博物館', '公民館', '区民館', '美術館', '記念館', '文書館',
    '休館', '開館', '公館', '旅館', '会館', '閉館', '分館', '本館', '旧館', '別館',
    '研究所', '停留所', '案内所', '事務所', '市役所', '区役所', '保健所', '実験所', '代官所', '保育所',
    '近所', '場所', '役所', '寝所', '別所', '住所', '個所', '札所', '入所', '出所',
    '人事院', '美容院', '四合院', '伝法院', '地理院', '参議院', '大学院', '衆議院', '密蔵院', '感化院',
    '病院', '子院', '入院', '退院', '法院', '寺院', '上院', '転院', '当院', '議院',
    '運動場', '事業場', '飛行場', '野球場', '駐車場', '練習場', '採取場', '駐輪場', '処理場',
    '登場', '会場', '市場', '球場', '上場', '式場', '戦場', '劇場', '開場', '工場', 
    '立場', '職場', '相場', '馬場', '現場', '夏場', '役場', '職場', '地場', '広場',
    '消防士', '弁護士', '保育士', '管理士', '技能士', '税理士', '会計士', '代議士',
    '騎士', '武士', '紳士', '義士', '兵士', '壮士', '浪士', '同士', '藩士', '範士',
    '研究者', '教育者', '分担者', '使用者', '雇用者', '債務者', '労働者', '筆記者', '消費者',
    '役者', '患者', '学者', '他者', '前者', '後者', '業者', '著者', '死者', '筆者',
    '会社員', '銀行員', '組合員', '通報員', '特派員', '作業員', '添乗員', '従業員',
    '一員', '全員', '役員', '委員', '職員', '会員', '船員', '人員', '総員', '行員',
    '小説家', '政治家', '音楽家', '写真家', '専門家', '脚本家', '研究家', '作曲家', '舞踊家', '投資家',
    '作家', '農家', '商家', '画家',
    '調理師', '美容師', '薬剤師', '絵図師', '説教師', '看護師', '宣教師', '連歌師', '助産師', '馬術師',
    '教師', '講師', '医師', '薬師', '牧師', '彫師', '漁師', '恩師', '法師', '老師'
]

In [ ]:
# составные единицы группы ваго с "префиксами"

kango_with_prefixes = [
    '無試験', '無計画', '無意味', '無関心', '無気力', '無条件', '無関係', '無防備', '無着陸飛行', '無教養',
    '無理', '無数', '無量', '無念', '無性', '無料', '無駄', '無視', '無効', '無職',
    '不必要', '不注意', '不親切', '不経済', '不勉強', '不自由', '不可能', '不平等', '不正確', '不参加', 
    '不要', '不便', '不調', '不安', '不満', '不明', '不平', '不足', '不和', '不利', '不正',
    '非公式', '非科学', '非論理', '非生産', '非人間', '非整数', '非提携', '非合法', '非産油', '非暴力',
    '非常', '非行', '非道', '非情', '非難', '非番', '非礼', '非核', '非極', '非人',
    '未完全', '未発表', '未使用', '未経験', '未編集', '未成年', '未分離', '未就学', '未曾有', '未完了',
    '未満', '未熟', '未知', '未婚', '未定', '未聞', '未完', '未進', '未然', '未詳',
    '総生産', '総人口', '総収入', '総輸入', '総選挙', '総人口', '総死亡', '総工費', '総領事', '総打数', 
    '総称', '総会', '総合', '総勢', '総数', '総括', '総額', '総員', '総則', '総評',
    '全科目', '全学生', '全生産', '全領域', '全産業', '全国民', '全日本', '全領土', '全人類', '全外貨',
    '全国', '全員', '全階', '全体', '全部', '全権', '全開', '全額', '全集', '全般',
    '諸問題', '諸事情', '諸費用', '諸欲求', '諸能力', '諸官庁', '諸情勢', '諸職人', '諸施策', '諸制度',
    '諸国', '諸島', '諸家', '諸州', '諸仏', '諸点', '諸元', '諸悪', '諸神', '諸法',
    '各会社', '各新聞', '各方面', '各分野', '各事業', '各大学', '各条項', '各学校', '各心理', '各効果',
    '各国', '各地', '各県', '各般', '各年', '各部', '各道', '各種', '各庁', '各月'
]

In [ ]:
# составные глаголы группы ваго, написание которых может варьироваться

wago_verbs_longer = [
    '取り組む', '勝ち越す', '呼び出す', '書き込む', '申し込む', '飛び込む', '差し出す', '振り込む'
]

wago_verbs_shorter = [
    '取組む', '勝越す', '呼出す', '書込む', '申込む', '飛込む', '差出す', '振込む'
]

In [ ]:
# составные существительные группы ваго, написание которых может варьироваться

wago_nouns_longer = [
    '取り組み', '受け付け', '申し込み', '組み立て', '割り引き', '立ち入れ', '売り上げ', '押し売り', '飛び込み', '差し出し'
]

wago_nouns_shorter = [
    '取組み', '受付け', '申込み', '組立て', '割引き', '立入れ', '売上げ', '押売り', '飛込み', '差出し'
]

wago_nouns_shortest = [
    '取組', '受付', '申込', '組立', '割引', '立入', '売上', '押売', '飛込', '差出'
]

In [ ]:
# составные существительные с сочинительной связью

noun_conjuncts = [
    '英独仏', '高中小', '雪月花', '上中下', '松竹梅', '親子孫', '年月日', 
    '父母', '子孫', '大小', '兄弟', '巨大', '姉妹', '危険', '白黒', '善良', '邪悪'
]

In [ ]:
# словарь составных единиц для поиска контекстов: ключи - типы составных единиц, значения - списки самих составных единиц

search_units = {
    'kango_with_postfixes': kango_with_postfixes,
    'kango_with_prefixes': kango_with_prefixes,
    'wago_verbs_longer': wago_verbs_longer,
    'wago_nouns_longer': wago_nouns_longer,
    'noun_conjuncts': noun_conjuncts
}

In [ ]:
# функция для получения файлов с контекстами по составной единице: 
# сначала выбираем параметр поиска - goiso, 
# вписываем составную единицу в поле поиска, 
# нажимаем "Поиск"; 
# если результаты поиска есть (try), 
# парсим необходимые данные, склеиваем в контекст, обрабатываем его и сохраняем в список контекстов, 
# список контекстов переробразовываем в dataframe и сохраняем в файл csv в нужную папку; 
# если результатов нет (except), печатаем составную единицу (их затем можно еще раз прогнать по поиску); 
# очищаем поле поиска и выжидаем некоторое время перед вводом следующей единицы

def raw_contexts_csv(search_unit, kind):
    select = Select(driver.find_element(By.CSS_SELECTOR, '#tab-pane-luw-search-by-form > div > div.context-cond.keyword-cond > div > div > div.panel-body > div > div > div:nth-child(1) > select'))
    select.select_by_visible_text('語彙素')
    luw_unit = driver.find_element(By.CSS_SELECTOR, '#tab-pane-luw-search-by-form > div > div.context-cond.keyword-cond > div > div > div.panel-body > div > div > div.form-group.tok-cond-item-ph > div:nth-child(2) > input')
    luw_unit.send_keys(search_unit)
    show_search_results = driver.find_element(By.CSS_SELECTOR, '#btn-group-search > button.btn-search.btn-search-related')
    show_search_results.click()

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#search-alert > strong.count')))

        pre_unit = driver.find_elements(By.CLASS_NAME, 'preceding-context')
        keyword = driver.find_elements(By.CLASS_NAME, 'keyword')
        post_unit = driver.find_elements(By.CLASS_NAME, 'following-context')
        contexts = []
        for i in range(len(pre_unit)):
            if pre_unit[i].text != '前文脈':
                if kind == 'wago_verbs_longer':
                    if re.fullmatch('[おこごそぞとどのほぼぽもろ]う', keyword[i].text[-2:]):
                        context = pre_unit[i].text + search_unit[:-1] + keyword[i].text[-2:] + post_unit[i].text
                    else:
                        context = pre_unit[i].text + search_unit[:-1] + keyword[i].text[-1] + post_unit[i].text
                else:
                    context = pre_unit[i].text + search_unit + post_unit[i].text
                clean_context = re.sub(r'\u3000', '', context)
                contexts.append(clean_context)
        contexts_df = pd.DataFrame(contexts)
        contexts_df.to_csv(f'raw_contexts/{kind}/{search_unit}.csv')

    except TimeoutException:
        print(search_unit)

    time.sleep(30)
    luw_unit.clear()

In [ ]:
driver = gs.Chrome()
wait = WebDriverWait(driver, 10)

In [ ]:
driver.get('https://chunagon.ninjal.ac.jp/auth/login?service=https%3A%2F%2Fchunagon.ninjal.ac.jp%2Fj_spring_cas_security_check')
wait.until(EC.presence_of_element_located((By.ID, 'submit')))

In [ ]:
username = driver.find_element(By.ID, 'username')
username.send_keys('моё_имя_пользователя')

password = driver.find_element(By.ID, 'password')
password.send_keys('мой_пароль')

login = driver.find_element(By.ID, 'submit')
login.click()

wait.until(EC.presence_of_element_located((By.LINK_TEXT, '現代日本語書き言葉均衡コーパス')))

In [ ]:
luw_search_button = driver.find_element(By.ID, 'tab-luw-search')
luw_search_button.click()
time.sleep(5)

In [ ]:
for kind, unit in search_units:
    raw_contexts_csv(unit, kind)

In [ ]:
driver.quit()

In [ ]:
for long_verb, short_verb in zip(wago_verbs_longer, wago_verbs_shorter):
    if f'{long_verb}.csv' in os.listdir('wago_verbs_longer'): # проверка на наличие файла с контекстами составной единицы
        file = pd.read_csv(f'wago_verbs_longer/{long_verb}.csv', encoding='utf-8', index_col=0)
        fixed_file = pd.DataFrame({
            '0': file['0'].str.replace(long_verb[:-1], short_verb[:-1])
        })
                
        fixed_file.to_csv(f'wago_verbs_shorter/{short_verb}.csv')

In [ ]:
for long_noun, shorter_noun, shortest_noun in zip(wago_nouns_longer, wago_nouns_shorter, wago_nouns_shortest):
    if f'{long_noun}.csv' in os.listdir('wago_nouns_longer'): # проверка на наличие файла с контекстами составной единицы
        file = pd.read_csv(f'wago_nouns_longer/{long_noun}.csv', encoding='utf-8', index_col=0)
        fixed_file_shorter_nouns = pd.DataFrame({
            '0': file['0'].str.replace(long_noun, shorter_noun)
        })
        fixed_file_shortest_nouns = pd.DataFrame({
            '0': file['0'].str.replace(long_noun, shortest_noun)
        })

        fixed_file_shorter_nouns.to_csv(f'wago_nouns_shorter/{shorter_noun}.csv')
        fixed_file_shortest_nouns.to_csv(f'wago_nouns_shortest/{shortest_noun}.csv')